# Linear Regression with Tensorflow

In [ ]:
# Import the libraries we'll use below.
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import tensorflow as tf

## Understanding the data
Goal: Train models using the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile)  from 1985 Ward's Automotive Yearbook that is part of the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets).



### Load the data
Load the data using the column names from [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile). We'll only use a few of the columns.

In [ ]:
# Provide the names for the feature columns since the CSV file with the data
# does not have a header row.
cols = ['symboling', 'losses', 'make', 'fuel-type', 'aspiration', 'num-doors',
        'body-style', 'drive-wheels', 'engine-location', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-type', 'num-cylinders',
        'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio',
        'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

# Load the data from a CSV file into a pandas dataframe. Remember that each row
# is an example and each column in a feature.
car_data = pd.read_csv(
    'https://storage.googleapis.com/ml_universities/cars_dataset/cars_data.csv',
    sep=',', names=cols, header=None, encoding='latin-1')

# Display applies built-in formatting for nicer printing, if available.
display(car_data)

### Randomize
With SGD (Stochastic Gradient Descent) for training, it is important that **each batch is a random sample of the data** so that the gradient computed is representative. The original data (above) appears sorted by *make* in alphabetic order.

In [ ]:
# We want to shuffle the order of the rows without touching the columns.
# First, we get a list of indices corresponding to the rows.
indices = np.arange(car_data.shape[0])
print('indices:', indices, '\n')

# Next, we shuffle the indices using np.random.permutation but set a random seed
# so that everyone gets the same results each time.
np.random.seed(0)
shuffled_indices = np.random.permutation(indices)
print('shuffled indices:', shuffled_indices, '\n')

# Finally, we use dataframe.reindex to change the ordering of the original
# dataframe.
car_data = car_data.reindex(shuffled_indices)
display(car_data)

# Note that this could be done in one fancy line:
# car_data = car_data.reindex(np.random.permutation(car_data.shape[0]))

### Feature selection
To keep things simple, I focused on just a few of the 26 columns. Since the values come as strings, we need to convert them to floats. Also, we remove examples (rows) that have some missing value(s) of the columns we care about.

In [ ]:
# Choose a subset of columns (these are all numeric).
columns = ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
car_data = car_data[columns]

# Convert strings to numeric values, coercing missing values to nan.
for column in columns:
  car_data[column] = pd.to_numeric(car_data[column], errors='coerce')

# The dropna function drops rows with missing value(s) by default.
car_data = car_data.dropna()

# This leaves us with 199 examples.
display(car_data)

### Train/Test split
Now that we've shuffled the order, let's split into portions for train and test easily. 

We're going to train models that **predict price from the other columns**, so we'll create separate variables for input and output data.


In [ ]:
# We'll use these input features.
features = ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']

# Use a ~80/20 train/test split.
car_train = car_data[:160]
car_test = car_data[160:]

# Create separate variables for features (inputs) and labels (outputs).
# We will be using these in the cells below.
car_train_features = car_train[features]
car_test_features = car_test[features]
car_train_labels = car_train['price']
car_test_labels = car_test['price']

# Confirm the data shapes are as expected.
print('train data shape:', car_train_features.shape)
print('train labels shape:', car_train_labels.shape)
print('test data shape:', car_test_features.shape)
print('test labels shape:', car_test_labels.shape)

---
### Baseline

Now that we have test data, we can evaluate a baseline. We'll use the average price of cars in the training set as our baseline model -- that is, the baseline always predicts the average price regardless of the input.

In [ ]:
# Implement baseline, average price
car_train_baseline = car_train_labels.mean()
car_train_baseline

In [ ]:
# compute Root mean square error (RMSE) for each model
print(
    'RMSE train baseline model:', np.sqrt(np.mean(np.square(car_train_labels - car_train_baseline)))
)

test_baseline_RMSE = np.sqrt(np.mean(np.square(car_test_labels - car_train_baseline)))

print('RMSE test baseline model:', test_baseline_RMSE)


The test RMSE is larger than the train RMSE. This makes sense since the baseline is an average of the prices training dataset.

---

### Feature histograms
It's hard to stare at a matrix of 160x5 numbers (the shape of our training data) and know what to make of it. Plotting feature histograms is a good way to start building intuition about the data. This gives us a sense of the distribution of each feature, but not how the features relate to each other.

In [ ]:
plt.figure(figsize=(15, 3))
for i in range(len(columns)):
  plt.subplot(1, 5, i+1)
  plt.hist(np.array(car_train[columns[i]]))
  plt.title(columns[i])
plt.show()

display(car_train.describe())

---
### Feature correlations

Using pandas [`corr()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) to print all the pairwise correlation coefficients for the columns (use the training data only). See also the [Wikipedia page on correlation](https://en.wikipedia.org/wiki/Correlation) for more background.

Then answer the following questions:

1. It appears that higher-priced cars have higher or lower fuel efficiency?
1. Which two features are likely to be most redundant?
1. Which feature is likely to be least useful for predicting price?

Extra (ungraded): try using [`sns.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html) to examine each pair of features.

In [ ]:
car_train.corr(method='pearson')

In [ ]:
sns.pairplot(car_train)


1. It appears that higher-priced cars have lower fuel efficiency, looking at both city-mpg and highway-mpg
2. City-mpg and highway-mpg appear to be most redudant as they are almost perfectly correlated, with a 0.97 corr coeff.
3. Peak-rpm appears to be least useful for predicting Price, given its small corr coeff.

---

## Tensorflow

Let's train a linear regression model using Tensorflow.

### Build a model
First, we build a *computational graph*, and then send data through it.

Here, we're using [`keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) to create a model layer.

In [ ]:
def build_model(num_features, learning_rate):
  """Build a TF linear regression model using Keras.

  Args:
    num_features: The number of input features.
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  # This is not strictly necessary, but each time you build a model, TF adds
  # new nodes (rather than overwriting), so the colab session can end up
  # storing lots of copies of the graph when you only care about the most
  # recent. Also, as there is some randomness built into training with SGD,
  # setting a random seed ensures that results are the same on each identical
  # training run.
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  # Build a model using keras.Sequential. While this is intended for neural
  # networks (which may have multiple layers), we want just a single layer for
  # linear regression.
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      units=1,                     # output dim
      input_shape=[num_features],  # input dim
      use_bias=True,               # use a bias (intercept) param
      kernel_initializer=tf.ones_initializer,  # initialize params to 1
      bias_initializer=tf.ones_initializer,    # initialize bias to 1
  ))

  # We need to choose an optimizer. We'll use SGD, which is actually mini-batch
  # SGD. We can specify the batch size to use for training later.
  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

  # Finally, we compile the model. This finalizes the graph for training.
  # We specify the MSE loss.
  model.compile(loss='mse', optimizer=optimizer)
  return model

After we've built a model, we can inspect the initial parameters (weights).

In [ ]:
# Build a model.
model = build_model(num_features=1, learning_rate=0.0001)

# Use get_weights() which returns lists of weights and biases for the layer.
weights, biases = model.layers[0].get_weights()
print('Weights:', weights)
print('Biases:', biases)

In [ ]:
# Build a model and look at the initial parameter values.
model = build_model(num_features=2, learning_rate=0.0001)
weights, biases = model.layers[0].get_weights()
print('Weights:', weights)
print('Biases:', biases)

### Train a model
Now let's actually train a model, initially with just 1 feature -- the horsepower. We use the `fit` function as it can take pandas DataFrame objects for input (x) and output (y). In addition, we can convert the return value into a DataFrame that tracks training metrics (in this case, training data loss and validation data loss) after each *epoch* (a full pass through the training data).

With SGD each time the model estimates the loss for the current weights, it randomly samples a batch of training examples to do so.

Finally, we'll reserve some more examples (taken out of the training set) as a *validation set*. We use this data to check for overfitting while training. 

In [ ]:
model = build_model(num_features=1, learning_rate=0.00001)

history = model.fit(
  x = car_train_features[['horsepower']],
  y = car_train_labels,
  validation_split=0.1,  # use 10% of the examples as a validation set
  epochs=5,
  batch_size=32,
  verbose=0)

# Convert the return value into a DataFrame so we can see the loss after each
# epoch. The history includes training data loss ('loss') and validation data
# loss ('val_loss').
history = pd.DataFrame(history.history)
display(history)

---
### Feature normalization

Apply mean and variance normalization to produce car_train_features_norm and car_test_features_norm.

In [ ]:
car_train_features_norm = (car_train_features - car_train_features.mean())/car_train_features.std()
car_test_features_norm = (car_test_features - car_train_features.mean())/car_train_features.std()

display(car_train_features_norm.describe())

---

### Training with features
Let's test out different sets of input features. To start, here's a simple function that plots train and validation set loss.

In [ ]:
def plot_loss(model, history):
  """Plot the loss after each training epoch."""
  # Convert the history object into a DataFrame.
  history = pd.DataFrame(history.history)

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(range(len(history)), history['loss'], marker='.', color='black')
  plt.plot(range(len(history)), history['val_loss'], marker='.', color='red')
  plt.legend(['train loss', 'validation loss'])
  plt.show()

  # Show the final train loss value and the learned model weights.
  print('Final train loss:', list(history['loss'])[-1])
  print('Final weights:', model.layers[0].get_weights())

  ## Show final val loss
  print('Final validation loss:', list(history['val_loss'])[-1])

---
### Adjusting learning rate

Retrain the model with different learning rates

In [ ]:
learning_rate_list = [0.001, 0.01, 0.1, 1]

# try learning rates
for rate in learning_rate_list:
  model = build_model(num_features=1, learning_rate=rate)

  history = model.fit(
    # use the normalized features prepared above
    x = car_train_features_norm[['horsepower']],
    y = car_train_labels,
    validation_split=0.1,
    epochs=150,
    batch_size=32,
    verbose=0)
  
  print("\n")
  print("Learning rate: ", rate)
  plot_loss(model, history)

---

A learning rate of 0.01 contributes the lowest validation loss after 150 epochs

### Adding features

Let's test 4 models:
1. features = horsepower
2. features = horsepower, peak-rpm
3. features = horsepower, peak-rpm, highway-mpg
4. features = horsepower, peak-rpm, highway-mpg, city-mpg

For consistency, we will use a batch size of 32, 150 epochs, and the best learning rate of 0.01 from above.

In [ ]:
# features = ['horsepower', 'peak-rpm', 'highway-mpg', 'city-mpg']

def run_experiment(features, learning_rate):
  model = build_model(len(features), learning_rate)

  history = model.fit(
    x = car_train_features_norm[features],
    y = car_train_labels,
    validation_split=0.1,
    epochs=150,
    batch_size=32,
    verbose=0)

  plot_loss(model, history)

  # Make predictions on test data
  test_loss = model.evaluate(car_test_features_norm[features],
                             car_test_labels,
                             verbose=0)
  test_rmse = np.sqrt(test_loss)
  print('Test rmse:', test_rmse)

  # return so we can store in list
  return test_rmse


In [ ]:
features = ['horsepower', 'peak-rpm', 'highway-mpg', 'city-mpg']

test_rmse_all = []

# loop through features list adding one more to model each time
for i in range(len(features)):
  print("\n")
  print("Features in model: ", features[:i+1])
  res = run_experiment(features[:i+1], 0.01)  

  test_rmse_all.append(res)

In [ ]:
# prepend Baseline model rmse to list
test_rmse_all.insert(0, test_baseline_RMSE)
test_rmse_all



Model | Test RMSE
--- | ---
Baseline | 
Horsepower | 
  + Peak-RPM | 
  + Highway-MPG | 
  + City-MPG | 

We can see below that the 4th model, which includes up to the Highway-MPG feature, has the lowest RMSE. This makes sense with our earlier intuition that the City-MPG feature is highly correlated with Highway-MPG and would likely not contribute meaningfully to our model.

In [ ]:
# Answer in table form

model_names = ["Baseline", "Horsepower", "+ Peak-RPM" , "+ Highway-MPG" , "+ City-MPG"]
model_output_df = pd.DataFrame(list(zip(model_names, test_rmse_all)),
                               columns =['Model', 'Test RMSE'])
model_output_df

---

## Takeways
* The **[Pandas](https://pandas.pydata.org/) library** is very useful for manipulating datasets and works well with numpy.
* Use a random split into train and test data and measure performance on the test data, starting from a simple **baseline**.
* Examine data using histograms and correlations to help build intuition before training any models.
* **Tensorflow** works by first building a **computational graph**; then, you can pass data through the graph to produce predictions, updating parameters via gradient descent in training mode; we use the **Keras API** to easily configure models.
* Training is often quite sensitive to the **learning rate** hyperparameter, and feature normalization is an important strategy.